### Unit 09: Hydrogen and Electricity Transport

#### See the exercises below

#### 1. Import FINE and further modules to run the model 

In [1]:
import FINE as fn
import geopandas as gpd
import pandas as pd
from os.path import dirname, abspath, join
import matplotlib.pyplot as plt
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2

#### 2. Set paths as input data for the model 

In [2]:
#set paths
cwd = os.getcwd()
data_dir = join(cwd, "data")

path_to_regions = join(data_dir, "regions", "region_shape_NER.shp")

path_to_pv_ts = join(data_dir, "sources", "solar_ts_NER.csv")
path_to_onshore_ts = join(data_dir, "sources", "onshore_ts_NER.csv")

path_to_pv_cap = join(data_dir, "sources", "solar_cap_NER.csv")
path_to_onshore_cap = join(data_dir, "sources", "onshore_cap_NER.csv")

path_to_el_dem = join(data_dir, "sinks", "electricity_dem_NER.xlsx")
path_to_h2_dem = join(data_dir, "sinks", "hydrogen_dem_NER.xlsx")

path_to_transmission_shp = join(data_dir, "transmission", "transmissions_NER.shp")
path_to_incidence_matrix = join(data_dir, "transmission", "incidence_matrix_NER.csv")
path_to_distance_matrix = join(data_dir, "transmission", "euclidean_distance_matrix_NER.csv")

#### 2.1 Set up the regions and the commodities of the model

In [3]:
locations_shape = gpd.read_file(path_to_regions)
locations = locations_shape.GID_1.to_list() #will be ["NER.1_1", "NER.2_1", ... "NER.8_1"]

commodities = {"electricity", "hydrogen_gas"}
commodityUnitsDict = {
                "electricity": r"GW$_{el}$",
                "hydrogen_gas": r"GW$_{H_{2},LHV}$",
            }

#### 3 Set up the energy system model class

In [4]:
#Set up esm Model

esM = fn.EnergySystemModel(
    locations=set(locations),
    commodities=commodities,
    numberOfTimeSteps=8760, #hours per year
    commodityUnitsDict=commodityUnitsDict,
    hoursPerTimeStep=1, #time step is one hour
    costUnit="1e9 Euro",
    lengthUnit="km",
    verboseLogLevel=0, #what is printed, just keep it
)


#### 4 Add the electricity "sources" to the model

In [50]:
#add PV
pv_time_series=pd.read_csv(path_to_pv_ts, index_col=[0]).reset_index(drop=True) #capacity factor [1]
pv_capacity_max=pd.read_csv(path_to_pv_cap, index_col=[0])['capacity_kW'] / 1e6 #capacity [GW]

esM.add(
    fn.Source(
        esM=esM, 
        name="PV", 
        commodity="electricity", 
        hasCapacityVariable=True,
        operationRateMax=pv_time_series,
        capacityMax=pv_capacity_max,
        investPerCapacity=0.450, #1e9EUR/GW, 2030
        opexPerCapacity=0.017 * 0.450, #1e9EUR/a
        interestRate=0.08,  #1
        economicLifetime=20, #a
        ),
)


#add Wind Onshore
onshore_time_series=pd.read_csv(path_to_onshore_ts, index_col=[0]).reset_index(drop=True) #capacity factor [1]
onshore_capacity_max=pd.read_csv(path_to_onshore_cap, index_col=[0])['capacity_kW'] / 1e6 #capacity [GW]

esM.add(
    fn.Source(
        esM=esM, 
        name="Onshore", 
        commodity="electricity", 
        hasCapacityVariable=True,
        operationRateMax=onshore_time_series,
        capacityMax=onshore_capacity_max,
        investPerCapacity=1.130, #1e9EUR/GW, 2030
        opexPerCapacity=0.025 * 1.130, #1e9EUR/a
        interestRate=0.08, #1
        economicLifetime=20, #y
        ),
)

 /storage/internal/home/s-ishmam/repos/fine/FINE/component.py:475: UserWarning:Component identifier PV already exists. Data will be overwritten.
 /storage/internal/home/s-ishmam/repos/fine/FINE/component.py:475: UserWarning:Component identifier Onshore already exists. Data will be overwritten.
 /storage/internal/home/s-ishmam/repos/fine/FINE/component.py:475: UserWarning:Component identifier electricity_demand already exists. Data will be overwritten.
 /storage/internal/home/s-ishmam/repos/fine/FINE/component.py:475: UserWarning:Component identifier hydrogen_demand already exists. Data will be overwritten.
 /storage/internal/home/s-ishmam/repos/fine/FINE/component.py:475: UserWarning:Component identifier electrolyzer already exists. Data will be overwritten.
 /storage/internal/home/s-ishmam/repos/fine/FINE/component.py:475: UserWarning:Component identifier Batteries already exists. Data will be overwritten.
 /storage/internal/home/s-ishmam/repos/fine/FINE/component.py:475: UserWarning:

#### 5 Add electrolyzer as conversion class 

In [ ]:
esM.add(
fn.Conversion(
    esM=esM,
    name="electrolyzer",
    physicalUnit=r"GW$_{el}$",
    commodityConversionFactors={"electricity": -1, "hydrogen_gas": 0.7},
    hasCapacityVariable=True,
    investPerCapacity=0.5,
    opexPerCapacity=0.5 * 0.025,
    interestRate=0.08,
    economicLifetime=10,
)
)


#### 6 Add Demand

Electricity & Hydrogen Demand

In [ ]:
#add Demands
electricity_demand_operationRateFix=pd.read_excel(path_to_el_dem, index_col=[0], engine="openpyxl")*3 #elec demand GW

esM.add(
    fn.Sink(
        esM=esM, 
        name="electricity_demand", 
        commodity="electricity",
        hasCapacityVariable=False, 
        operationRateFix=electricity_demand_operationRateFix, #GW
    ),
)

#add Demands
hydrogen_demand_operationRateFix=pd.read_excel(path_to_h2_dem, index_col=[0], engine="openpyxl")*3 #elec demand GW

esM.add(
    fn.Sink(
        esM=esM, 
        name="hydrogen_demand", 
        commodity="hydrogen_gas",
        hasCapacityVariable=False, 
        operationRateFix=hydrogen_demand_operationRateFix, #GW
    ),
)

#### 7 Add storage

Batteries

In [ ]:
# add batteries
esM.add(
    fn.Storage(
        esM= esM,
        name= "Batteries", 
        commodity="electricity",
        hasCapacityVariable= True, 
        chargeEfficiency=0.95, #1
        dischargeEfficiency=0.95, #1
        cyclicLifetime=10000, #1
        selfDischarge=4.230E-05, #1
        chargeRate=1, #C-Rate: 1/h
        dischargeRate=1, #C-Rate: 1/h
        doPreciseTsaModeling= False,
        investPerCapacity=0.17511, #1e9EUR/GW, 2030
        opexPerCapacity= 0.02 * 0.17511, #1e9EUR/a
        interestRate=0.08, #1
        economicLifetime=20, #a
    ),
)

Hydrogen

In [ ]:
#add hydrogen storage
esM.add(
    fn.Storage(
        esM=esM,
        name=f"hydrogen_storage",
        commodity=f"hydrogen_gas",
        hasCapacityVariable=True,
        chargeEfficiency=1,
        cyclicLifetime=10000,
        dischargeEfficiency=1,
        selfDischarge=0, # 1% per day
        chargeRate=1,
        dischargeRate=1,
        doPreciseTsaModeling=False,
        investPerCapacity=0.0007, #0.70EUR/kWh = 0.0007 BEUR/GWh
        opexPerCapacity=0.02 * 0.0007,
        interestRate=0.08,
        economicLifetime=40,
    )
)

#### Task 1: Load the distance and incidence matrices

In [46]:
# distance_matrix = ...
# incidence_matrix = ...



#### Task 2 (a) : Add hydrogen pipelines as a transmission component to the model

In [47]:
# esM.add(
# fn.Transmission(
#    name="hydrogen_pipelines"
#    ...
#     )
# )

#### Task 2 (b) : Add electricity grid as a transmission component to the model

In [49]:
# esM.add(
# fn.Transmission(
#    name="electricity_grid",
#     ...
#     )
# )

In [18]:
# Temporal Aggregation
esM.aggregateTemporally(numberOfTypicalPeriods=7, 
                        segmentation=True,
                        numberOfSegmentsPerPeriod=8)


Clustering time series data with 7 typical periods and 24 time steps per period 
further clustered to 8 segments per period...
		(0.6410 sec)



In [34]:
#RUN the model:
print('Optimize')
esM.optimize(
    timeSeriesAggregation=True,
    optimizationSpecs="",
    solver="glpk"
)
print('Optimization done!')

Optimize
Time series aggregation specifications:
Number of typical periods:7, number of time steps per period:24, number of segments per period:8

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1091 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0173 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1226 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(1.0601 sec)

Declaring shared potential constraint...
		(0.0003 sec)

Declaring linked component quantity constraint...
		(0.0000 sec)

Declaring commodity balances...
		(0.0933 sec)

		(0.0004 sec)

Declaring objective function...
		(0.0446 sec)

GLPSOL: GLPK LP/MIP

 /storage/internal/home/s-ishmam/repos/fine/FINE/storage.py:1771: UserWarning:Charge and discharge at the same time for component Batteries
 /storage/internal/home/s-ishmam/repos/fine/FINE/storage.py:1771: UserWarning:Charge and discharge at the same time for component hydrogen_storage


for StorageModel ...       (1.7062sec)
		(3.4898 sec)

Optimization done!


#### Task 3 (a) : Get the optimization summary of the TransmissionModel

In [52]:
# esM.getOptimizationSummary(.......) 

#### Task 3 (b) : Display the total optimal capacity of the electricity grid and the hydrogen grid

In [61]:
# esM.getOptimizationSummary(.......).loc[('electricity_grid', .............)]

In [69]:
# esM.getOptimizationSummary(.......).loc[('hydrogen_pipelines', .............)]

#### Task 3 (c) : Display the total annual cost (TAC) of the electricity grid and the hydrogen grid

In [62]:
# esM.getOptimizationSummary(.......).loc[('electricity_grid', .............)]

In [70]:
# esM.getOptimizationSummary(.......).loc[('hydrogen_pipelines', .............)]

#### Task 4 (a) : Plot the optimal capacities of the electricity grid

In [67]:
# fig, ax = fn.plotLocations(path_to_regions, indexColumn="index", crs='epsg:3857')
# fig, ax = fn.plotTransmission(
#     esM, "electricity_grid", .....
# )

#### Task 4 (b) : Plot the optimal capacities of the hydrogen grid¶


In [68]:
# fig, ax = fn.plotLocations(path_to_regions, indexColumn="index", crs='epsg:3857')
# fig, ax = fn.plotTransmission(
#     esM, "hydrogen_pipelines", .....
# )